In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weather/weather.csv


# Libraries

In [2]:
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Constants

In [3]:
API_KEY = '902635857ca292a7bc2c9a8ac5aa773b'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

# Weather Data

In [4]:
def get_5day_weather_forecast(city):
    url = f"{BASE_URL}forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    forecast_data = []

    for forecast in data['list']:
        forecast_data.append({
            'city': data['city']['name'],
            'date': forecast['dt_txt'],
            'current_temp': round(forecast['main']['temp']),
            'feels_like': round(forecast['main']['feels_like']),
            'temp_min': round(forecast['main']['temp_min']),
            'temp_max': round(forecast['main']['temp_max']),
            'humidity': round(forecast['main']['humidity']),
            'description': forecast['weather'][0]['description'],
            'country': data['city']['country'],
            'wind_gust_dir': forecast['wind']['deg'],
            'pressure': forecast['main']['pressure'],
            'wind_gust_speed': forecast['wind']['speed']
        })
    
    return forecast_data

# Preparing ANN

In [5]:
def prepare_data_for_ann(data):
    features = ['Temp', 'Humidity', 'WindGustSpeed', 'Pressure']
    X = data[features].values
    y = data['Temp'].values
    
    # Normalize data
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, scaler

# Read Historical Data

In [6]:
def read_historical_data(filename):
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.drop_duplicates()
    return df

# Build Model

In [7]:
def build_ann_model(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu')) 
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])
    return model

# Train Model

In [8]:
def train_ann_model(X, y):
    model = build_ann_model(X.shape[1])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
    return model

# Predict Model

In [9]:
def predict_future_temp(model, current_data):
    prediction = model.predict(current_data)
    return prediction

# Evaluating Model

In [10]:
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    return mse, mae, r2

# Weather Analysis Function

In [11]:
def weather_view():
    city = "Wardha"
    forecast = get_5day_weather_forecast(city)
    historical_data = read_historical_data("/kaggle/input/weather/weather.csv")

    X, y, scaler = prepare_data_for_ann(historical_data)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    ann_model = train_ann_model(X_train, y_train)
    mse, mae, r2 = evaluate_model(ann_model, X_test, y_test)
    print(f"Model Evaluation:")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"R-squared: {r2}")

    predicted_temp = None
    for i, day in enumerate(forecast):
        if i == 0: 
            current_data = np.array([[day['current_temp'], 
                                      day['humidity'], 
                                      day['wind_gust_speed'], 
                                      day['pressure']]])
        else:
            predicted_temp_value = predicted_temp[0][0] if predicted_temp is not None else day['current_temp']
            
            current_data = np.array([[predicted_temp_value, 
                                      day['humidity'], 
                                      day['wind_gust_speed'], 
                                      day['pressure']]])

        current_data_scaled = scaler.transform(current_data)

        predicted_temp = predict_future_temp(ann_model, current_data_scaled)

        print(f"Date: {day['date']}")
        print(f"City: {day['city']}, {day['country']}")
        print(f"Current Temp: {day['current_temp']}°C")
        print(f"Minimum Temperature: {day['temp_min']}°C")
        print(f"Maximum Temperature: {day['temp_max']}°C")
        print(f"Humidity: {day['humidity']}%")
        print(f"Weather Description: {day['description']}")
        print(f"Predicted Future Temperature for Next Day: {round(predicted_temp[0][0], 2)}°C")
        print("-" * 50)


In [12]:
weather_view()

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 408.4898 - mae: 18.9886 - mse: 408.4898 - val_loss: 428.7179 - val_mae: 19.6159 - val_mse: 428.7179
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 384.5277 - mae: 18.4671 - mse: 384.5277 - val_loss: 405.3370 - val_mae: 19.0178 - val_mse: 405.3370
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 379.5042 - mae: 18.1117 - mse: 379.5042 - val_loss: 371.0484 - val_mae: 18.1015 - val_mse: 371.0484
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 329.8367 - mae: 16.8155 - mse: 329.8367 - val_loss: 318.9731 - val_mae: 16.6125 - val_mse: 318.9731
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 283.9748 - mae: 15.3441 - mse: 283.9748 - val_loss: 242.9289 - val_mae: 14.1621 - val_mse: 242.9289
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 191.8973 - mae: 12.1402 - mse: 191.8973 - val_loss: 147.1426 - val_mae: 10.2749 - val_mse: 147.1426
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 103.4725 - mae: 8.